In [2]:
# %matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import glob
import warnings
from IPython.display import display

In [3]:
# read whole year data
allFiles = glob.glob("2014-2017/*/*.csv")
df = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0,encoding="utf-8")
    list_.append(df)
df = pd.concat(list_)

df.shape

(179179, 49)

In [5]:
df.columns = map(str.lower, df.columns)
df.columns = df.columns.str.replace('_', ' ')

In [6]:
df.dtypes

tracking id                    int64
marketing code                object
original code                 object
marketing name                object
classification                object
app source                    object
web source                    object
enquired                      object
first call                    object
last access                   object
last access by                object
next contact date             object
decision                      object
appointed                     object
submitted                     object
lead approval date            object
reporting approval date       object
potential settlement date     object
settled                       object
lender                        object
hlc                           object
rejected reason               object
loan amount                   object
enquiry status                object
hot transfer                  object
held                          object
held by                       object
p

In [3]:
df.drop_duplicates(inplace=True)

In [4]:
df.columns = map(str.lower, df.columns)
df.columns = df.columns.str.replace('_', ' ')

In [5]:
target = 'enquiry status';
df = df[~df[target].isin(['In Progress','Just Received','On Hold'])]

ValueError: could not convert string to float: '500001-$1,000,000'

In [6]:
# invalid_post_code = ['QLD','victoria',"270-1176","VIC","2575b","6110r","2150s"]
# if 'post code' in df.columns: 
#     df = df[~df['post code'].isin(invalid_post_code)]

In [7]:
invalid_loan_amount = ['500001-$1000000',
              '300001-$500000',
              '0-$300000',
              '250000 - 300000',
              '250000-350000',
              '2600 monthly',
              'not_sure',
              '1000,001+',
              '9999-',
              'I50000',
              '1.5 M',
              '1000001+',
              '9999-',
              '80-90k']

test = df[df['loan amount'].isin(invalid_loan_amount)]
df = df[~df['loan amount'].isin(invalid_loan_amount)]

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameColumnsTransform(BaseEstimator, TransformerMixin):
    def __init__(self, seperator):
        self.seperator = seperator
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        x.columns = map(str.lower, df.columns)
        return X.columns.str.replace(self.seperator, ' ')

In [10]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]

In [11]:
class DataFrameClean(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer

selected_features = ['classification','enquired',
                     'loan amount','loan reason','property use']

prepocess_pipeline = Pipeline([
        ("transform_data_column", DataFrameColumnsTransform("_")),
        ("select_data", DataFrameSelector(selected_features))
    ])